In [1]:
import textract
import os, sys
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers.util import semantic_search, pytorch_cos_sim
import torch
from pymilvus import MilvusClient

sys.path.append(os.path.abspath('../'))

from app.config import config


def read_pdf_file(file_path):
    return textract.process(file_path).decode('utf-8', 'ignore')#.replace("\n", ' ')

def get_all_pdf_files(storage_pdf_path):
    return [file for file in os.listdir(storage_pdf_path) if file.endswith('.pdf')]

def store_file_in_db(pdf_file, storage_pdf_path, text_splitter, embeddings, client):
    text = read_pdf_file(storage_pdf_path + pdf_file)

    chunks = text_splitter.split_text(text)

    embedded_chunks = embeddings.embed_documents(chunks)

    res = client.query(
        collection_name="oab_documents",
        output_fields=["count(*)"]
    )

    lenght = res[0]['count(*)']

    data = []

    for i in range(len(embedded_chunks)):
        data.append({
            "id": lenght + i,
            "vector": embedded_chunks[i],
            "text": chunks[i],
            "subject": pdf_file
        })

    client.insert(
        collection_name="oab_documents",
        data=data
    )


/home/chapolin/aio/langchain-llm-example/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [16]:
# config['embeddings_model'] = 'neuralmind/bert-base-portuguese-cased'

config

{'app_env': 'prod',
 'llm_api_model': 'meta/llama3-70b-instruct',
 'llm_api_key': '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',
 'embeddings_model': 'neuralmind/bert-base-portuguese-cased'}

In [3]:
storage_pdf_path = '../storage/pdf/'

pdf_files = get_all_pdf_files(storage_pdf_path)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    keep_separator=False,
    separators=[
        ".\n",
        # ".",
    ]
)

embeddings = HuggingFaceEmbeddings(model_name=config['embeddings_model'])


No sentence-transformers model found with name neuralmind/bert-base-portuguese-cased. Creating a new one with mean pooling.


In [206]:
text = read_pdf_file(storage_pdf_path + pdf_files[1])
# text
chunks = text_splitter.split_text(text)

chunks

['Ordem dos Advogados do Brasil\nConselho Seccional do Rio Grande do Sul\nRua Washington Luiz, 1110\n90010-460 Porto Alegre – RS\nTelefone: 51 3287.1800 - http://www.oabrs.org.br\n\nOfício n° 000226/2021/GP\nProtocolo n° 21.0000.2021.000226-3\n\nPorto Alegre, 22 de fevereiro de 2021',
 '\nExcelentíssimo Senhor\nDesembargador Voltaire de Lima Moraes\nPresidente do Tribunal de Justiça do Estado\nE-mail: presidencia@tj.rs.gov.br; presidencia@tjrs.jus.br\nRM/US\n\nAssunto: Ato nº 01/2021-1VP/CGJ, Ato nº 030/2020-CGJ – Bandeira Preta – Prazos Processos\nEletrônicos – Atendimento',
 'Caro Presidente,\n\n1',
 'Ao cumprimentá-lo, considerando a Declaração de Emergência em Saúde\nPública de Importância Internacional pela Organização Mundial da Saúde (OMS), em\ndecorrência da infecção pela COVID-19 (Coronavírus) e sua classificação como pandemia,\nbem como a recente edição do Decreto Estadual nº 55.764, de 20 de fevereiro de 2021, vimos a\npresença de Vossa Excelência expor e requerer providênci

In [4]:
client = MilvusClient("../storage/milvus.db")

client.create_collection(
  collection_name="oab_documents",
  dimension=embeddings.client[1].word_embedding_dimension
)

In [182]:
# client.drop_collection(
#   collection_name="oab_documents",
# )

In [5]:
client.delete(
    collection_name="oab_documents",
    filter="id not in [-1]"
)

for pdf_file in pdf_files:
    store_file_in_db(pdf_file, storage_pdf_path, text_splitter, embeddings, client)

# store_file_in_db(pdf_files[0], storage_pdf_path, text_splitter, embeddings, client)
# store_file_in_db(pdf_files[1], storage_pdf_path, text_splitter, embeddings, client)
# store_file_in_db(pdf_files[3], storage_pdf_path, text_splitter, embeddings, client)

In [17]:
client.delete(
    collection_name="oab_documents",
    filter="subject == 'file_598e37ec8db3e.pdf'"
)

[35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 21

In [210]:
store_file_in_db(pdf_files[4], storage_pdf_path, text_splitter, embeddings, client)

In [162]:
# embeddings.embed_query('qual o motivo da suspensao dos servicos eletronicos?')

In [19]:
client.search(
  collection_name="oab_documents",
  data=[embeddings.embed_query('qual o motivo da suspensao dos servicos eletronicos?')],
#   filter="subject == 'history'",
  limit=10,
  output_fields=["text", "subject"],
)

data: ["[{'id': 3717, 'distance': 0.6548517942428589, 'entity': {'text': 'ORDEM DOS ADVOGADOS DO BRASIL DE GOIÁS. CF Valentina Jungmann apresenta\\nprojeto que estabelece paridade entre homens e mulheres nas eleições da OAB. Disponível em:\\nhttps://www.oabgo.org.br/oab/noticias/reivindicacao/cf-valentina-jungmann-apresentaprojeto-que-estabelece-paridade-entre-homens-e-mulheres-nas-eleicoes-da-oab/. Acesso em\\n13 abr. 2021', 'subject': 'file_611c23465ce9f.pdf'}}, {'id': 3718, 'distance': 0.5918455719947815, 'entity': {'text': 'ORDEM DOS ADVOGADOS DO BRASIL DO RIO GRANDE DO SUL. Artigo do\\npresidente da OAB/RS publicado no Estadão: Não apenas no Carnaval. Disponível em:\\nhttps://www.oabrs.org.br/noticias/artigo-presidente-oabrs-publicado-no-estadao--nao-apenascarnaval/41305. Acesso em: 13 abr. 2021', 'subject': 'file_611c23465ce9f.pdf'}}, {'id': 3723, 'distance': 0.5847059488296509, 'entity': {'text': 'SECCHI, Leonardo et al. Mandatos coletivos e compartilhados: inovação na represent

In [176]:
pytorch_cos_sim(
    embeddings.embed_query('qual o motivo da suspensao dos servicos eletronicos?'),
    [
        embeddings.embed_query('qual o motivo da suspensao dos servicos eletronicos?'),
        embeddings.embed_query('Ola mundo'),
        embeddings.embed_query('sei ano mano'),
        embeddings.embed_query('Não existem motivos para que os prazos eletrônicos tenham suspensão, pois seu cumprimento depende única e exclusivamente de manejo virtual, seja por parte dos servidores, magistrados,  auxiliares  da  justiça,  advogados  e  advogadas.  Salientamos  que  eventual impossibilidade de cumprimento, se justificada, poderá ser objeto de possível análise do caso concreto.'),
        embeddings.embed_query('possam garantir o efetivo cumprimento do princípio constitucional de amplo acesso à justiça\n(art. 5º, XXXV, da Constituição Federal).\n4.\nNão existem motivos para que os prazos eletrônicos tenham suspensão, pois seu\ncumprimento depende única e exclusivamente de manejo virtual, seja por parte dos servidores,\nmagistrados, auxiliares da justiça, advogados e advogadas. Salientamos que eventual\nimpossibilidade de cumprimento, se justificada, poderá ser objeto de possível análise do caso\nconcreto.\n5.\nAinda, considerando as inúmeras demandas recebidas por parte da advocacia\ngaúcha acerca das negativas de atendimento, seja por parte dos servidores, seja por parte de\nmagistrados e desembargadores, pedimos que Vossa Excelência cumpra os termos da'),
        embeddings.embed_query("3.\nComo já defendido pela OAB/RS, e sobretudo no atual momento de crise\nvivenciado, não se nega a necessidade de cautela com relação às Comarcas de Bandeira\nPreta, sendo prudente um atendimento diferenciado. Porém, diante da prática de atividade\nremota e virtual adquirida ao longo de quase um ano de Pandemia, ainda, aliada às normativas\nemanadas pelo Conselho Nacional de Justiça em âmbito nacional, se faz necessária uma\nreflexão, especialmente ao fato de que cabe ao Judiciário implementar mecanismos que\npossam garantir o efetivo cumprimento do princípio constitucional de amplo acesso à justiça\n(art. 5º, XXXV, da Constituição Federal).\n4.\nNão existem motivos para que os prazos eletrônicos tenham suspensão, pois seu\ncumprimento depende única e exclusivamente de manejo virtual, seja por parte dos servidores,\nmagistrados, auxiliares da justiça, advogados e advogadas. Salientamos que eventual\nimpossibilidade de cumprimento, se justificada, poderá ser objeto de possível análise do caso"),
        embeddings.embed_query("possam garantir o efetivo cumprimento do princípio constitucional de amplo acesso à justiça\n(art. 5º, XXXV, da Constituição Federal).\n4.\nNão existem motivos para que os prazos eletrônicos tenham suspensão, pois seu\ncumprimento depende única e exclusivamente de manejo virtual, seja por parte dos servidores,\nmagistrados, auxiliares da justiça, advogados e advogadas. Salientamos que eventual\nimpossibilidade de cumprimento, se justificada, poderá ser objeto de possível análise do caso\nconcreto.\n5.\nAinda, considerando as inúmeras demandas recebidas por parte da advocacia\ngaúcha acerca das negativas de atendimento, seja por parte dos servidores, seja por parte de\nmagistrados e desembargadores, pedimos que Vossa Excelência cumpra os termos da"),
        embeddings.embed_query('e ao atendimento. 3. Como já defendido pela OAB/RS, e sobretudo no atual momento de crise vivenciado, não se nega a necessidade de cautela com relação às Comarcas de Bandeira Preta, sendo prudente um atendimento diferenciado. Porém, diante da prática de atividade remota e virtual adquirida ao longo de quase um ano de Pandemia, ainda, aliada às normativas emanadas pelo Conselho Nacional de Justiça em âmbito nacional, se faz necessária uma reflexão, especialmente ao fato de que cabe ao Judiciário implementar mecanismos que possam garantir o efetivo cumprimento do princípio constitucional de amplo acesso à justiça (art. 5º, XXXV, da Constituição Federal). 4. Não existem motivos para que os prazos eletrônicos tenham suspensão, pois seu'),
        embeddings.embed_query('de Justiça em âmbito nacional, se faz necessária uma reflexão, especialmente ao fato de que cabe ao Judiciário implementar mecanismos que possam garantir o efetivo cumprimento do princípio constitucional de amplo acesso à justiça (art. 5º, XXXV, da Constituição Federal). 4. Não existem motivos para que os prazos eletrônicos tenham suspensão, pois seu cumprimento depende única e exclusivamente de manejo virtual, seja por parte dos servidores, magistrados, auxiliares da justiça, advogados e'),
        embeddings.embed_query( 'existem motivos para que os prazos eletrônicos tenham suspensão, pois seu cumprimento depende única e exclusivamente de manejo virtual, seja por parte dos servidores, magistrados, auxiliares da'),
        embeddings.embed_query("que também pode ser levado em consideração para a manutenção da motivação do aluno no âmbito da área jurídica é o fato de utilizar toda sua experiência adquirida ao longo da vida durante o ensino-aprendizagem em sala de aula. Assim, o docente antes de iniciar o estudo de novos conteúdos teóricos pode estabelecer os objetivos do ensinoaprendizagem e alinhá-los as experiências e exigências de vida dos alunos. Ou seja, deve-se capitar as expectativas de conhecimentos que os alunos possuem em relação a disciplina a ser ministrada, para, posteriormente, elaborar plano de aula e estratégias de ensina de acordo com elas. Ou seja, primeiramente os alunos apresentam seus conhecimentos sobre o ensino teórico, bem como as situações vivenciadas por eles no seu cotidiano. Após tais informações, o docente realizada uma seleção das experiências e expectativas dos alunos em relação a disciplina a ser ministra, para, posteriormente, elaborar plano de aula e estratégias de ensino de acordo com elas."),
        embeddings.embed_query('sei ano mano'),
    ]
)

tensor([[1.0000, 0.3455, 0.2627, 0.5832, 0.5640, 0.5501, 0.5640, 0.5428, 0.5688,
         0.6131, 0.4968, 0.2627]])

In [21]:
# from langchain_community.vectorstores import FAISS

# db = FAISS.

# from sentence_transformers import util


pytorch_cos_sim(embeddings.embed_query('qual o motivo da suspensao dos servicos eletronicos?'), embedded_chunks)

tensor([[0.3531, 0.3812, 0.5984, 0.3690, 0.5662, 0.5235, 0.3006]])

In [30]:

semantic_search(
    torch.FloatTensor(embeddings.embed_query('qual o motivo da suspensao dos servicos eletronicos?')),
    torch.FloatTensor(embedded_chunks),
    top_k=3
)

[[{'corpus_id': 2, 'score': 0.5983752608299255},
  {'corpus_id': 4, 'score': 0.5661625862121582},
  {'corpus_id': 5, 'score': 0.5235135555267334}]]

In [31]:
print(chunks[2], end="\n-------\n")
print(chunks[4], end="\n-------\n")
print(chunks[5], end="\n-------\n")
# print(chunks[6], end="\n-------\n")

possam garantir o efetivo cumprimento do princípio constitucional de amplo acesso à justiça
(art. 5º, XXXV, da Constituição Federal).
4.
Não existem motivos para que os prazos eletrônicos tenham suspensão, pois seu
cumprimento depende única e exclusivamente de manejo virtual, seja por parte dos servidores,
magistrados, auxiliares da justiça, advogados e advogadas. Salientamos que eventual
impossibilidade de cumprimento, se justificada, poderá ser objeto de possível análise do caso
concreto.
5.
Ainda, considerando as inúmeras demandas recebidas por parte da advocacia
gaúcha acerca das negativas de atendimento, seja por parte dos servidores, seja por parte de
magistrados e desembargadores, pedimos que Vossa Excelência cumpra os termos da
-------
6.
Observamos que o atendimento poderá ser feito de forma virtual e por meio de
qualquer tipo de ferramenta tecnológica, permitindo assim que os servidores, mesmo em
trabalho remoto, possam atender à advocacia utilizando qualquer tipo de ferramen

In [33]:
client = MilvusClient("../storage/milvus.db")

client.create_collection(
  collection_name="demo_collection",
  dimension=5
)

In [87]:
client.insert(
  collection_name="demo_collection",
  data=[
    {"id": 0, "vector": [0.1, 0.2, 0.3, 0.2, 0.1], "text": "AI was proposed in 1956.", "subject": "history"},
    {"id": 1, "vector": [0.1, 0.2, 0.3, 0.3, 0.1], "text": "Alan Turing was born in London.", "subject": "history"},
    {"id": 2, "vector": [0.1, 0.2, 0.3, 0.2, 0.4], "text": "Computational synthesis with AI algorithms predicts molecular properties.", "subject": "biology"},
  ]
)

{'insert_count': 3, 'ids': [0, 1, 2], 'cost': 0}

In [88]:
client.search(
  collection_name="demo_collection",
  data=[[0.1, 0.2, 0.3, 0.2, 0.4]],
#   filter="subject == 'history'",
  limit=2,
  output_fields=["text", "subject"],
)

data: ["[{'id': 2, 'distance': 1.0, 'entity': {'text': 'Computational synthesis with AI algorithms predicts molecular properties.', 'subject': 'biology'}}, {'id': 0, 'distance': 0.8655784130096436, 'entity': {'text': 'AI was proposed in 1956.', 'subject': 'history'}}]"] 

In [85]:
client.delete(
  collection_name="demo_collection",
  ids=[0,1,2]
)

{'delete_count': 0, 'cost': 0}

In [86]:
res = client.query(
    collection_name="demo_collection",
    output_fields=["count(*)"]
)

print(res[0]['count(*)'])

0
